In [1]:
import pandas as pd
import numpy as np
from os import listdir

The cells below important the weather station coordinates file to get the formatted names, and initializes some lists for easy cleaning in the later functions.

In [2]:
names_df = pd.read_csv('weather_stations_coordinates.csv')
names = list(names_df['0'])
names.sort()

In [3]:
path = ['weather/','/days_by_year']

In [4]:
months_sel = ['Year', 'Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp (°C)',
              'Total Rain (mm)', 'Total Snow (cm)', 'Total Precip (mm)']

In [5]:
months_rename = ['Month', 'Year', 'Day', 'Mean Max Temp', 'Mean Min Temp', 'Mean Temp',
              'Total Rain', 'Total Snow', 'Total Precip']

In [6]:
years_sel = ['Year', 'Month', 'Mean Max Temp (°C)', 'Mean Min Temp (°C)', 'Mean Temp (°C)',
              'Total Rain (mm)', 'Total Snow (cm)', 'Total Precip (mm)']

In [7]:
years_rename = ['Year', 'Month', 'Mean Max Temp', 'Mean Min Temp', 'Mean Temp',
              'Total Rain', 'Total Snow', 'Total Precip']

These two functions are for whether the data is stored with each row as a day-months by year, or month-year in one file. Using the lists above, the functions correctly get the desired weather data, and then renames it appropriately. 

In [8]:
def get_merged(town, files):
    output = []
    for file in files:
        df = pd.read_csv(town + '/days_by_year/' + file)
        df = df[months_sel]
        dfm = df.groupby('Month').mean().reset_index()
        dfm.columns = months_rename
        dfm = dfm.drop(['Day','Total Rain','Total Snow', 'Total Precip'],axis=1)
        dfs = df.groupby('Month').sum().reset_index()
        dfs.columns = months_rename
        dfs = dfs.drop('Day',axis=1)
        dfs = dfs[['Total Rain','Total Snow', 'Total Precip']]
        dff = pd.concat([dfm, dfs], axis=1)
        output.append(dff)
    merged = pd.concat(output, axis=0)
    merged = merged[['Year', 'Month', 'Mean Max Temp', 'Mean Min Temp', 'Mean Temp',
           'Total Rain', 'Total Snow', 'Total Precip']]
    merged = merged.dropna()
    return merged

In [9]:
def get_year(town, files):
    df = pd.read_csv(town + '/months_by_years/' + files[0])
    df = df[years_sel]
    df.columns = years_rename
    return df

Using the above functions, the cell below loops through all weather stations and outputs a clean weather data file in each folder.

In [11]:
towns = [x for x in listdir() if x.find('.') == -1]
for town in towns:
    folders = [x for x in listdir(town) if x.find('.') == -1]
    output = pd.DataFrame()
    for folder in folders:
        files = [x for x in listdir(town + '/' + folder) if x[-4:] == '.csv']
        if folder  == 'days_by_year':
            merged = get_merged(town, files)
            output = pd.concat([output, merged], axis=0)
        elif folder == 'months_by_years':
            cleaned = get_year(town, files)
            output = pd.concat([output, cleaned], axis=0)
    output = output.sort_values(['Year','Month'])
    output = output.dropna()
    output.to_csv(town + '/' + town + '_cleaned.csv', index=False)

Using the clean weather data from each weather station folder as generated above, this cell concatenates all the data into one large file.

In [17]:
towns = [x for x in listdir() if x.find('.') == -1]
towns.sort()
data_list = []
for i in range(len(towns)):
    temp = pd.read_csv(towns[i] + '/' + towns[i] + '_cleaned.csv')
    temp['Weather City'] = names[i]
    data_list.append(temp)
compiled_data = pd.concat(data_list)
compiled_data = compiled_data[['Weather City', 'Year', 'Month', 'Mean Max Temp', 'Mean Min Temp', 'Mean Temp', 'Total Rain', 'Total Snow', 'Total Precip']]

In [18]:
compiled_data

,Weather City,Year,Month,Mean Max Temp,Mean Min Temp,Mean Temp,Total Rain,Total Snow,Total Precip
0,Bella Coola,1983.0,2,7.400000,0.400000,3.900000,74.0,1.8,75.0
1,Bella Coola,1983.0,3,12.400000,-0.800000,5.800000,37.5,0.0,37.5
2,Bella Coola,1983.0,4,16.300000,2.600000,9.500000,9.5,0.0,9.5
3,Bella Coola,1983.0,5,19.300000,7.000000,13.200000,36.0,0.0,36.0
4,Bella Coola,1983.0,6,17.900000,10.900000,14.400000,89.4,0.0,89.4
...,...,...,...,...,...,...,...,...,...
728,Williams Lake,2021.0,8,21.985714,9.157143,15.600000,28.5,0.0,32.9
729,Williams Lake,2021.0,9,16.906667,5.833333,11.396667,36.3,0.0,36.3
730,Williams Lake,2021.0,10,8.710345,-0.058621,4.355172,24.1,0.6,24.6
731,Williams Lake,2021.0,11,3.866667,-3.359259,0.259259,14.4,14.0,26.6


In [19]:
compiled_data.to_csv('clean_weather.csv', index=False)